# Day 14

## Part 1
Given is a list of masks and values to write to locations in memory.

Line by line, each time we see a mask, that is the new mask, and that mask will overwrite the 36-bit binary version of the value being assigned to memory.  If the mask has a 0 or 1, that is the new bit in the value, it the mask has an X, leave the original bit.

After masking all the values and assigning them to memory, what is the sum of all values assigned to memory?

In [1]:
f = open('day14.txt')
prog = f.read().splitlines()
f.close()

In [2]:
prog[:10]

['mask = X100110110X011000101000101XX11001X11',
 'mem[5201] = 1838761',
 'mem[32099] = 25747352',
 'mem[36565] = 72187',
 'mem[31494] = 369864',
 'mem[17260] = 3138',
 'mem[64903] = 91484814',
 'mask = 0X00100101000XX0011110X10110X100X010',
 'mem[54866] = 120526',
 'mem[57614] = 430839']

In [3]:
def applyMask(mask, val):
    newVal = bin(val)[2:]  # convert to binary
    newVal = '0'*(36-len(newVal)) + newVal   # add leading 0s
    maskedVal = ''.join([newVal[i] if mask[i] == 'X' else mask[i] for i in range(36)])   # apply the mask
    return int(maskedVal, 2)  # return the base_10 value to be written to memory

applyMask('XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X', 0)

64

In [4]:
known = {}
for i, line in enumerate(prog):
    if line[:4] == 'mask':
        mask = line.split(' = ')[1]
    else:
        loc, val = line.split(' = ')
        loc = int(loc[4:-1])
        val = int(val)
        newVal = applyMask(mask, val)
        known[loc] = newVal

In [5]:
sum(known.values())

6559449933360

## Part 2

Now, the masks are modifying the locations in memory, not the values.  

    0: leave that bit alone
    1: change the mem bit to 1
    X: leave as X, can take on either 0 or 1, all possible combos of mem address should take on the new val

#### Example

    mask = 000000000000000000000000000000X1001X
    mem[42] = 100
    mask = 00000000000000000000000000000000X0XX
    mem[26] = 1
    
    should give answer of 208
    
In the first mem write above, mask has 2 X's, so there are 4 possible combos the X's could take on (00, 01, 10, 11), and we want to write the value of 100 to all 4 of them.  The second write has a mask with 3 X's, so there are 8 different locations we'll write the value of 1 to memory.

In [6]:
def applyMask2(mask, memloc):
    # convert loc to binary with leading 0s
    loc = bin(memloc)[2:]
    loc = '0'*(36-len(loc)) + loc
    
    # apply mask to the loc, leaving in Xs
    maskedloc = ''
    for i in range(36):
        curr = mask[i]
        if curr == '0':
            maskedloc += loc[i]
        elif curr == '1':
            maskedloc += '1'
        else:
            maskedloc += 'X'
            
    # get all possible locs (all binary combos possible for Xs)
    locs = []
    xcount = maskedloc.count('X')
    for i in range(2**xcount):
        ml = maskedloc[:]  # make a copy of masked loc so we can modify it
        insert = bin(i)[2:]  # create binary string of vals to be inserted
        insert = '0'*(xcount-len(insert)) + insert  # add leading zeros
        for j in range(len(insert)):
            ml = ml.replace('X', insert[j], 1)  # replace Xs one at a time
        locs.append(int(ml, 2))
        
    # return the list of locations in memory we need to write to
    return list(set(locs))

applyMask2('X000000000000000000000000000000000000', 42)

[42, 34359738410]

In [7]:
def getAnswer(data):
    known2 = {}
    for i, line in enumerate(data):
        if line[:4] == 'mask':
            mask = line.split(' = ')[1]
        else:
            loc, val = line.split(' = ')
            loc = int(loc[4:-1])
            val = int(val)
            newlocs = applyMask2(mask, loc)
            #print(newlocs)
            for nl in newlocs:
                known2[nl] = val
    return sum(known2.values())

In [8]:
sample1 = ['mask = 000000000000000000000000000000X1001X',
'mem[42] = 100',
'mask = 00000000000000000000000000000000X0XX',
'mem[26] = 1']

In [9]:
getAnswer(sample1)

208

In [10]:
getAnswer(prog)

3369767240513